code from: https://keras.io/examples/nlp/lstm_seq2seq/
Author: @fchollet

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "data/rus.txt"

In [4]:
# Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # we use "tab" as the start sequence character
    # for the targets, and "\n" as the end sequence character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples: ", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples:  10000
Number of unique input tokens: 73
Number of unique output tokens: 85
Max sequence length for inputs: 14
Max sequence length for outputs: 60


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

# Build the model

In [8]:
# Defines an input sequence and processes it
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# we discard encoder_outputs and keep only the states
encoder_states = state_h, state_c

# set up the decoder using encoder_states as the initial state
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and return internal states as well. We don't use the
# return states in the training model, but we will use them in inference
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# encoder_input_data & decoder_input_data into target_data
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Train the model

In [9]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
125/125 [==============================] - 19s 128ms/step - loss: 1.3112 - accuracy: 0.7472 - val_loss: 0.8474 - val_accuracy: 0.7679
Epoch 2/100
125/125 [==============================] - 14s 115ms/step - loss: 0.7309 - accuracy: 0.8090 - val_loss: 0.8328 - val_accuracy: 0.7920
Epoch 3/100
125/125 [==============================] - 16s 126ms/step - loss: 0.6145 - accuracy: 0.8355 - val_loss: 0.6830 - val_accuracy: 0.8315
Epoch 4/100
125/125 [==============================] - 17s 133ms/step - loss: 0.5372 - accuracy: 0.8521 - val_loss: 0.5607 - val_accuracy: 0.8386
Epoch 5/100
125/125 [==============================] - 14s 116ms/step - loss: 0.4843 - accuracy: 0.8600 - val_loss: 0.5203 - val_accuracy: 0.8502
Epoch 6/100
125/125 [==============================] - 15s 118ms/step - loss: 0.4557 - accuracy: 0.8676 - val_loss: 0.4969 - val_accuracy: 0.8574
Epoch 7/100
125/125 [==============================] - 14s 115ms/step - loss: 0.4336 - accuracy: 0.8732 - val_loss: 0.4902 -

Epoch 57/100
125/125 [==============================] - 14s 116ms/step - loss: 0.0699 - accuracy: 0.9780 - val_loss: 0.5490 - val_accuracy: 0.8883
Epoch 58/100
125/125 [==============================] - 14s 114ms/step - loss: 0.0709 - accuracy: 0.9775 - val_loss: 0.5507 - val_accuracy: 0.8879
Epoch 59/100
125/125 [==============================] - 14s 113ms/step - loss: 0.0688 - accuracy: 0.9778 - val_loss: 0.5591 - val_accuracy: 0.8877
Epoch 60/100
125/125 [==============================] - 14s 114ms/step - loss: 0.0664 - accuracy: 0.9788 - val_loss: 0.5651 - val_accuracy: 0.8873
Epoch 61/100
125/125 [==============================] - 14s 113ms/step - loss: 0.0652 - accuracy: 0.9791 - val_loss: 0.5642 - val_accuracy: 0.8882
Epoch 62/100
125/125 [==============================] - 14s 114ms/step - loss: 0.0625 - accuracy: 0.9800 - val_loss: 0.5709 - val_accuracy: 0.8877
Epoch 63/100
125/125 [==============================] - 14s 113ms/step - loss: 0.0613 - accuracy: 0.9804 - val_loss: 0

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


## Run inference (sampling)

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]